In [36]:
import pandas as pd
import numpy as np
import streamlit as st

In [37]:
brasileirao = pd.read_excel('../data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [38]:
brasileirao['Score_m']

0      2.0
1      0.0
2      2.0
3      2.0
4      2.0
      ... 
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
Name: Score_m, Length: 380, dtype: float64

In [39]:
brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].count()

228

In [40]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
225,2023,23,RED,2.0,GRE,0.0
226,2023,23,SAN,0.0,CRU,3.0
227,2023,23,PAL,1.0,GOI,0.0
228,2023,23,VAS,4.0,FLU,2.0


In [41]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J' : [jog],
        '4-V' : [vit],
        '5-E' : [emp],
        '6-D' : [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,23,16,3,4,39.0,14.0,51,25.0
0,PAL,23,12,8,3,38.0,17.0,44,21.0
9,GRE,23,12,4,7,38.0,31.0,40,7.0
8,FLA,23,11,6,6,36.0,30.0,39,6.0
4,RED,23,10,9,4,31.0,21.0,39,10.0
10,FLU,23,11,5,7,33.0,26.0,38,7.0
2,CAP,23,10,7,6,35.0,26.0,37,9.0
5,FOR,23,10,5,8,27.0,22.0,35,5.0
6,CAM,23,9,7,7,26.0,19.0,34,7.0
11,CUI,23,8,5,10,23.0,29.0,29,-6.0


In [74]:
tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time']

3     BOT
0     PAL
9     GRE
8     FLA
4     RED
10    FLU
2     CAP
5     FOR
6     CAM
11    CUI
13    CRU
14    INT
12    SAO
7     COR
19    BAH
18    GOI
16    SAN
15    VAS
1     AME
17    CFC
Name: 1-Time, dtype: object

In [43]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,23,12,8,3,38.0,17.0,44,21.0,1
1,AME,22,4,5,13,26.0,47.0,17,-21.0,0
2,CAP,23,10,7,6,35.0,26.0,37,9.0,3
3,BOT,23,16,3,4,39.0,14.0,51,25.0,1
4,RED,23,10,9,4,31.0,21.0,39,10.0,3
5,FOR,23,10,5,8,27.0,22.0,35,5.0,3
6,CAM,23,9,7,7,26.0,19.0,34,7.0,3
7,COR,23,6,9,8,27.0,29.0,27,-2.0,2
8,FLA,23,11,6,6,36.0,30.0,39,6.0,3
9,GRE,23,12,4,7,38.0,31.0,40,7.0,3


In [44]:
print(kmeans.labels_)

[1 0 3 1 3 3 3 2 3 3 3 4 2 2 4 0 0 0 4 4]


In [45]:
kmeans.cluster_centers_

array([[ 18.        ,   4.25      ,   5.25      ,  12.75      ,
        -18.75      ],
       [ 47.5       ,  14.        ,   5.5       ,   3.5       ,
         23.        ],
       [ 28.        ,   6.66666667,   8.        ,   8.        ,
          1.33333333],
       [ 37.42857143,  10.42857143,   6.14285714,   6.42857143,
          7.28571429],
       [ 27.        ,   6.75      ,   6.75      ,   9.5       ,
         -6.5       ]])

In [46]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[ 18.     4.25   5.25  12.75 -18.75]
1
[47.5 14.   5.5  3.5 23. ]
2
[28.          6.66666667  8.          8.          1.33333333]
3
[37.42857143 10.42857143  6.14285714  6.42857143  7.28571429]
4
[27.    6.75  6.75  9.5  -6.5 ]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,18.000000,4.250000,5.250000,12.750000,-18.750000
1,1,47.500000,14.000000,5.500000,3.500000,23.000000
2,2,28.000000,6.666667,8.000000,8.000000,1.333333
3,3,37.428571,10.428571,6.142857,6.428571,7.285714
4,4,27.000000,6.750000,6.750000,9.500000,-6.500000


In [47]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
1,1,47.500000,14.000000,5.500000,3.500000,23.000000
3,3,37.428571,10.428571,6.142857,6.428571,7.285714
2,2,28.000000,6.666667,8.000000,8.000000,1.333333
4,4,27.000000,6.750000,6.750000,9.500000,-6.500000
0,0,18.000000,4.250000,5.250000,12.750000,-18.750000


In [48]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
1,1,47.500000,14.000000,5.500000,3.500000,23.000000,Título
3,3,37.428571,10.428571,6.142857,6.428571,7.285714,Libertadores
2,2,28.000000,6.666667,8.000000,8.000000,1.333333,Sul-Americana
4,4,27.000000,6.750000,6.750000,9.500000,-6.500000,Limbo
0,0,18.000000,4.250000,5.250000,12.750000,-18.750000,Rebaixamento


In [49]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
1,1,Título
3,3,Libertadores
2,2,Sul-Americana
4,4,Limbo
0,0,Rebaixamento


In [50]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,23,12,8,3,38.0,17.0,44,21.0,1
1,AME,22,4,5,13,26.0,47.0,17,-21.0,0
2,CAP,23,10,7,6,35.0,26.0,37,9.0,3
3,BOT,23,16,3,4,39.0,14.0,51,25.0,1
4,RED,23,10,9,4,31.0,21.0,39,10.0,3
5,FOR,23,10,5,8,27.0,22.0,35,5.0,3
6,CAM,23,9,7,7,26.0,19.0,34,7.0,3
7,COR,23,6,9,8,27.0,29.0,27,-2.0,2
8,FLA,23,11,6,6,36.0,30.0,39,6.0,3
9,GRE,23,12,4,7,38.0,31.0,40,7.0,3


In [51]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,23,16,3,4,39.000000,14.000000,51,25.000000,1
0,PAL,23,12,8,3,38.000000,17.000000,44,21.000000,1
9,GRE,23,12,4,7,38.000000,31.000000,40,7.000000,3
8,FLA,23,11,6,6,36.000000,30.000000,39,6.000000,3
4,RED,23,10,9,4,31.000000,21.000000,39,10.000000,3
10,FLU,23,11,5,7,33.000000,26.000000,38,7.000000,3
2,CAP,23,10,7,6,35.000000,26.000000,37,9.000000,3
5,FOR,23,10,5,8,27.000000,22.000000,35,5.000000,3
6,CAM,23,9,7,7,26.000000,19.000000,34,7.000000,3
11,CUI,23,8,5,10,23.000000,29.000000,29,-6.000000,4


In [52]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [53]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [54]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [55]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [56]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [57]:
tabela["cluster_pred"] = pred_logreg

In [58]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [59]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [60]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,23,12,8,3,38.0,17.0,44,21.0,1,1,0.0000,0.4915,0.0300,0.4711,0.0074
1,AME,22,4,5,13,26.0,47.0,17,-21.0,0,0,0.8527,0.0000,0.0125,0.0034,0.1314
2,CAP,23,10,7,6,35.0,26.0,37,9.0,3,3,0.0032,0.1202,0.1094,0.6774,0.0898
3,BOT,23,16,3,4,39.0,14.0,51,25.0,1,1,0.0000,0.6784,0.0002,0.3209,0.0005
4,RED,23,10,9,4,31.0,21.0,39,10.0,3,3,0.0005,0.1656,0.1923,0.5840,0.0576
5,FOR,23,10,5,8,27.0,22.0,35,5.0,3,3,0.0244,0.0634,0.0530,0.7094,0.1498
6,CAM,23,9,7,7,26.0,19.0,34,7.0,3,3,0.0107,0.0619,0.1892,0.5729,0.1653
7,COR,23,6,9,8,27.0,29.0,27,-2.0,2,2,0.0290,0.0031,0.5666,0.1062,0.2951
8,FLA,23,11,6,6,36.0,30.0,39,6.0,3,3,0.0031,0.1268,0.0386,0.7620,0.0694
9,GRE,23,12,4,7,38.0,31.0,40,7.0,3,3,0.0048,0.1374,0.0098,0.8028,0.0452


In [61]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.0000,0.4915,0.0300,0.4711,0.0074
1,AME,0.8527,0.0000,0.0125,0.0034,0.1314
2,CAP,0.0032,0.1202,0.1094,0.6774,0.0898
3,BOT,0.0000,0.6784,0.0002,0.3209,0.0005
4,RED,0.0005,0.1656,0.1923,0.5840,0.0576
5,FOR,0.0244,0.0634,0.0530,0.7094,0.1498
6,CAM,0.0107,0.0619,0.1892,0.5729,0.1653
7,COR,0.0290,0.0031,0.5666,0.1062,0.2951
8,FLA,0.0031,0.1268,0.0386,0.7620,0.0694
9,GRE,0.0048,0.1374,0.0098,0.8028,0.0452


In [62]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.000000,0.491500,0.030000,0.471100,0.007400
1,AME,0.852700,0.000000,0.012500,0.003400,0.131400
2,CAP,0.003200,0.120200,0.109400,0.677400,0.089800
3,BOT,0.000000,0.678400,0.000200,0.320900,0.000500
4,RED,0.000500,0.165600,0.192300,0.584000,0.057600
5,FOR,0.024400,0.063400,0.053000,0.709400,0.149800
6,CAM,0.010700,0.061900,0.189200,0.572900,0.165300
7,COR,0.029000,0.003100,0.566600,0.106200,0.295100
8,FLA,0.003100,0.126800,0.038600,0.762000,0.069400
9,GRE,0.004800,0.137400,0.009800,0.802800,0.045200


In [63]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,0.0,3.0,3.0,1.0,0.0,3.0,1.0,3.0,3.0,0.0,0.0,3.0
2,0.0,0.0,6.0,1.0,3.0,0.0,3.0,3.0,1.0,3.0,6.0,4.0,3.0,3.0,4.0,4.0,4.0,1.0,3.0,4.0
3,0.0,3.0,9.0,4.0,3.0,1.0,3.0,6.0,1.0,3.0,6.0,7.0,3.0,6.0,7.0,7.0,4.0,4.0,4.0,4.0
4,0.0,6.0,12.0,4.0,6.0,1.0,4.0,9.0,4.0,3.0,7.0,8.0,3.0,7.0,7.0,10.0,5.0,4.0,7.0,5.0
5,1.0,6.0,15.0,7.0,9.0,2.0,4.0,9.0,4.0,6.0,10.0,9.0,3.0,7.0,7.0,13.0,6.0,7.0,8.0,6.0
6,1.0,6.0,15.0,10.0,12.0,2.0,5.0,12.0,4.0,9.0,13.0,10.0,6.0,8.0,7.0,14.0,7.0,10.0,9.0,6.0
7,4.0,7.0,18.0,13.0,12.0,2.0,5.0,12.0,7.0,12.0,13.0,10.0,7.0,11.0,7.0,15.0,10.0,11.0,12.0,6.0
8,4.0,7.0,21.0,14.0,12.0,3.0,8.0,13.0,8.0,13.0,13.0,13.0,7.0,14.0,10.0,16.0,13.0,11.0,15.0,6.0
9,7.0,8.0,21.0,17.0,15.0,3.0,8.0,13.0,11.0,16.0,16.0,14.0,7.0,17.0,11.0,19.0,13.0,12.0,15.0,6.0


In [64]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,0.0,...,3.0,1.0,0.0,3.0,1.0,3.0,3.0,0.0,0.0,3.0
1,2,0.0,0.0,6.0,1.0,3.0,0.0,3.0,3.0,1.0,...,6.0,4.0,3.0,3.0,4.0,4.0,4.0,1.0,3.0,4.0
2,3,0.0,3.0,9.0,4.0,3.0,1.0,3.0,6.0,1.0,...,6.0,7.0,3.0,6.0,7.0,7.0,4.0,4.0,4.0,4.0
3,4,0.0,6.0,12.0,4.0,6.0,1.0,4.0,9.0,4.0,...,7.0,8.0,3.0,7.0,7.0,10.0,5.0,4.0,7.0,5.0
4,5,1.0,6.0,15.0,7.0,9.0,2.0,4.0,9.0,4.0,...,10.0,9.0,3.0,7.0,7.0,13.0,6.0,7.0,8.0,6.0
5,6,1.0,6.0,15.0,10.0,12.0,2.0,5.0,12.0,4.0,...,13.0,10.0,6.0,8.0,7.0,14.0,7.0,10.0,9.0,6.0
6,7,4.0,7.0,18.0,13.0,12.0,2.0,5.0,12.0,7.0,...,13.0,10.0,7.0,11.0,7.0,15.0,10.0,11.0,12.0,6.0
7,8,4.0,7.0,21.0,14.0,12.0,3.0,8.0,13.0,8.0,...,13.0,13.0,7.0,14.0,10.0,16.0,13.0,11.0,15.0,6.0
8,9,7.0,8.0,21.0,17.0,15.0,3.0,8.0,13.0,11.0,...,16.0,14.0,7.0,17.0,11.0,19.0,13.0,12.0,15.0,6.0
9,10,8.0,9.0,24.0,18.0,16.0,4.0,9.0,14.0,12.0,...,17.0,14.0,8.0,17.0,14.0,22.0,14.0,13.0,15.0,6.0


In [65]:
from sklearn.linear_model import LinearRegression

In [66]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

ValueError: Input y contains NaN.

In [ ]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.72
1,BAH,0.96
2,BOT,2.40
3,CAM,1.36
4,CAP,1.56
5,CFC,0.87
6,COR,1.20
7,CRU,1.22
8,CUI,1.58
9,FLA,1.85


In [ ]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,20,10,7,3,36.0,17.0,37,19.0,3,3,0.0000,0.3583,0.0055,0.4957,0.1405,PAL,1.62
1,AME,20,2,5,13,20.0,44.0,11,-24.0,0,0,0.8482,0.0002,0.1418,0.0000,0.0098,AME,0.72
2,CAP,20,9,5,6,29.0,23.0,32,6.0,1,1,0.0051,0.5040,0.0699,0.0940,0.3271,CAP,1.56
3,BOT,20,15,3,2,35.0,11.0,48,24.0,3,3,0.0000,0.2790,0.0001,0.7171,0.0039,BOT,2.40
4,RED,20,8,8,4,27.0,21.0,32,6.0,1,4,0.0009,0.3896,0.0696,0.1321,0.4077,RED,1.70
5,FOR,20,8,5,7,22.0,19.0,29,3.0,4,4,0.0203,0.3472,0.1527,0.0380,0.4417,FOR,1.32
6,CAM,20,7,6,7,22.0,18.0,27,4.0,4,4,0.0197,0.2126,0.1857,0.0261,0.5559,CAM,1.36
7,COR,20,6,7,7,21.0,22.0,25,-1.0,4,4,0.0285,0.1245,0.3026,0.0100,0.5345,COR,1.20
8,FLA,20,10,5,5,34.0,26.0,35,8.0,1,1,0.0011,0.6033,0.0279,0.1754,0.1924,FLA,1.85
9,GRE,20,10,4,6,29.0,25.0,34,4.0,1,1,0.0045,0.6505,0.0480,0.0992,0.1978,GRE,1.81


In [ ]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,20,15,3,2,35.000000,11.000000,48,24.000000,3,3,0.000000,0.279000,0.000100,0.717100,0.003900,BOT,2.400000,42.500000,12.500000,5.000000,2.500000,21.500000,2.010000
0,PAL,20,10,7,3,36.000000,17.000000,37,19.000000,3,3,0.000000,0.358300,0.005500,0.495700,0.140500,PAL,1.620000,42.500000,12.500000,5.000000,2.500000,21.500000,2.010000
8,FLA,20,10,5,5,34.000000,26.000000,35,8.000000,1,1,0.001100,0.603300,0.027900,0.175400,0.192400,FLA,1.850000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
10,FLU,20,10,4,6,28.000000,20.000000,34,8.000000,1,1,0.003100,0.611400,0.036100,0.133400,0.216000,FLU,1.550000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
9,GRE,20,10,4,6,29.000000,25.000000,34,4.000000,1,1,0.004500,0.650500,0.048000,0.099200,0.197800,GRE,1.810000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
2,CAP,20,9,5,6,29.000000,23.000000,32,6.000000,1,1,0.005100,0.504000,0.069900,0.094000,0.327100,CAP,1.560000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
4,RED,20,8,8,4,27.000000,21.000000,32,6.000000,1,4,0.000900,0.389600,0.069600,0.132100,0.407700,RED,1.700000,33.400000,9.400000,5.200000,5.400000,6.400000,1.694000
5,FOR,20,8,5,7,22.000000,19.000000,29,3.000000,4,4,0.020300,0.347200,0.152700,0.038000,0.441700,FOR,1.320000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333
11,CUI,20,8,4,8,21.000000,23.000000,28,-2.000000,4,4,0.072700,0.312700,0.232700,0.016400,0.365600,CUI,1.580000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333
12,SAO,20,7,7,6,24.000000,19.000000,28,5.000000,4,4,0.007300,0.232200,0.150200,0.041500,0.568800,SAO,1.470000,27.000000,7.000000,6.000000,7.000000,2.000000,1.358333


In [ ]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
19,20,11,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,17


In [ ]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,48.0,0.0,48.0
15,PAL,37.0,0.0,37.0
9,FLA,35.0,0.0,35.0
10,FLU,34.0,0.0,34.0
13,GRE,34.0,0.0,34.0
16,RED,32.0,0.0,32.0
4,CAP,32.0,0.0,32.0
11,FOR,29.0,0.0,29.0
18,SAO,28.0,0.0,28.0
8,CUI,28.0,0.0,28.0
